In [ ]:
import pandas as pd
import numpy as np
import nltk
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.combine import SMOTEENN
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, matthews_corrcoef,balanced_accuracy_score, f1_score,recall_score,precision_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, average_precision_score, precision_recall_curve
from imblearn.under_sampling import EditedNearestNeighbours
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import RandomOverSampler


from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

In [ ]:

file_path = 'Jeyaraman_2020_cleaned.csv'
# file_path = 'Radjenovic_2013_cleaned.csv'
# file_path = 'Smid_2020_cleaned.csv'
# file_path = 'Hall_2012_cleaned.csv'

df = pd.read_csv(file_path, delimiter=',')
df = df.dropna(axis=0)
df_sample = df.copy()
df_sample = df.reset_index(drop=True)



In [ ]:
# import pandas as pd
# import random
# from nltk.corpus import wordnet

# def get_synonyms(word):
#     synonyms = set()
#     for syn in wordnet.synsets(word):
#         for lemma in syn.lemmas():
#             synonym = lemma.name().replace('_', ' ')
#             synonyms.add(synonym)
#     if word in synonyms:
#         synonyms.remove(word)
#     return list(synonyms)

# def synonym_replacement(sentence, n):
#     words = sentence.split()
#     new_words = words.copy()
#     random_word_list = list(set([word for word in words if word.isalpha()]))
#     random.shuffle(random_word_list)
#     num_replaced = 0
#     for random_word in random_word_list:
#         synonyms = get_synonyms(random_word)
#         if len(synonyms) >= 1:
#             synonym = random.choice(list(synonyms))
#             new_words = [synonym if word == random_word else word for word in new_words]
#             num_replaced += 1
#         if num_replaced >= n: # only replace up to n words
#             break

#     sentence = ' '.join(new_words)
#     return sentence

# def augment_text(df, minority_class, augment_by=0.9):
#     minority_df = df[df['label_included'] == minority_class]
    
#     n_minority = len(minority_df)
#     n_augmentations = int(n_minority * augment_by)
    
#     augmented_texts = []
#     for _ in range(n_augmentations):
#         original_text = random.choice(minority_df['Corpus'].tolist())
#         augmented_text = synonym_replacement(original_text, n=1) # You can adjust n for more replacements
#         augmented_texts.append(augmented_text)
    
#     # Create a DataFrame for augmented minority samples
#     augmented_df = pd.DataFrame(augmented_texts, columns=['Corpus'])
#     augmented_df['label_included'] = minority_class
    
#     return augmented_df

# # Assuming your minority class is identified, for example, as 1
# df_augmented = augment_text(df, minority_class=1, augment_by=0.5)


# df_sample = pd.concat([df, df_augmented], ignore_index=True)

# # Shuffle the dataframe to mix original and augmented examples (optional)
# df_sample = df_sample.sample(frac=1).reset_index(drop=True)




In [ ]:
X = df_sample['Corpus']
y= df_sample['label_included']
# Encode labels to numeric values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
class_counts = df_augmented['label_included'].value_counts()
print(class_counts)
class_counts = df_sample['label_included'].value_counts()
print(class_counts)

X = df_sample['Corpus']
y= df_sample['label_included']
# Encode labels to numeric values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
NUM_FOLDS = 5
RANDOM_STATE = 42
scaler = StandardScaler(with_mean=False)
param_grid = {
    'penalty': ['l1', 'l2'],  # Regularization penalty
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Inverse of regularization strength
    'solver': ['liblinear', 'saga'],  # Algorithm to use in the optimization problem
    'max_iter': [100, 200, 300]  # Maximum number of iterations
}

In [ ]:
# smote_enn = SMOTEENN()
logistic_classifier = LogisticRegression()
grid_search = GridSearchCV(estimator=logistic_classifier, param_grid=param_grid, scoring='accuracy', cv=NUM_FOLDS, n_jobs=-1, verbose=2)

In [ ]:


def train_evaluate_logistic_regression(X_train, y_train, X_test, y_test, param_grid, kfold):
    # Data preprocessing
    tfidf_vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(2, 3), max_df=0.7)
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    X_test_tfidf = tfidf_vectorizer.transform(X_test)

    X_train_tfidf_std = scaler.fit_transform(X_train_tfidf)
    X_test_tfidf_std = scaler.transform(X_test_tfidf)



    # Logistic Regression model training and evaluation
    grid_search = GridSearchCV(estimator=LogisticRegression(), param_grid=param_grid, scoring='accuracy', cv=kfold, n_jobs=-1, verbose=2)
    grid_search.fit(X_train_tfidf_std, y_train)

    best_params = grid_search.best_params_
    best_lr_classifier = grid_search.best_estimator_

    best_lr_classifier.fit(X_train_tfidf_std, y_train)
    y_pred = best_lr_classifier.predict(X_test_tfidf_std)

    balanced_acc = balanced_accuracy_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    return best_params, balanced_acc, mcc, f1, recall,precision, tfidf_vectorizer,best_lr_classifier


In [ ]:
# Inside the loop, call the modified function:
stratified_kfold = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=RANDOM_STATE)

# Lists to store evaluation metrics
balanced_acc_scores = []
mcc_scores = []
f1_scores = []
recall_scores = []
precision_scores =[]



for fold_idx, (train_index, test_index) in enumerate(stratified_kfold.split(X, y_encoded), 1):
    X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_test_fold = y_encoded[train_index], y_encoded[test_index]

    # Train and evaluate Logistic Regression model
    best_params, balanced_acc_fold, mcc_fold, f1_fold, recall_fold, precision_fold,tfidf_vectorizer, best_lr_classifier = train_evaluate_logistic_regression(
        X_train_fold, y_train_fold, X_test_fold, y_test_fold, param_grid, stratified_kfold
    )

    # Append scores to lists
    balanced_acc_scores.append(balanced_acc_fold)
    mcc_scores.append(mcc_fold)
    f1_scores.append(f1_fold)
    recall_scores.append(recall_fold)
    precision_scores.append(precision_fold)
    

    # Print results for the fold
    print(f"Fold {fold_idx}:")
    print(f"  Best Hyperparameters: {best_params}")
    print(f"  Balanced Accuracy: {balanced_acc_fold:.2f}")
    print(f"  F1-Score: {f1_fold:.2f}")
    print(f"  Matthew's Correlation Coefficient: {mcc_fold:.2f}")
    print(f"  Recall: {recall_fold:.2f}")
    print(f"  Precision: {precision_fold:.2f}")


In [ ]:
print("Average Scores tfidf:")
print(f"{np.mean(mcc_scores):.2f}")
print(f"{np.mean(balanced_acc_scores):.2f}")
print(f"{np.mean(f1_scores):.2f}")
print(f"{np.mean(precision_scores):.2f}")
print(f"{np.mean(recall_scores):.2f}")

In [ ]:
import numpy as np

# ...other imports and model code as necessary...

# Assuming 'texts' is a list of strings you want to explain
for i, text in enumerate(texts):
    # Create an explanation for the i-th text
    exp = explainer.explain_instance(text, predict_proba, num_features=20, labels=[1], num_samples=50)
    exp.show_in_notebook(text=True)
    
    # Save the explanation figure if needed
    fig = exp.as_pyplot_figure()
    fig.savefig(f'lime_explanation_{i}.png', bbox_inches='tight')  # Saves each explanation as a separate PNG file

    # Convert the explanation to a list of tuples and then to a DataFrame
    feature_names = exp.as_list(label=1)
    df = pd.DataFrame(feature_names, columns=['Feature', 'Weight'])

    # Display the DataFrame for this explanation
    print(df)

# Code to remove important features according to LIME
# Extract only the feature names from the explanations
important_features = [feature for feature, weight in exp.as_list(label=1)]

# This assumes you have a way to map from feature names back to your feature matrix columns
# Here, we mock this by assuming 'feature_mapping' is a dictionary that maps from feature names to column indexes
# feature_mapping = {'feature_name': column_index, ...}
important_indices = [feature_mapping[feature] for feature in important_features if feature in feature_mapping]

# Reduce X by deleting the columns corresponding to important features
reduced_X = np.delete(X, important_indices, axis=1)

# Function 'evaluate_model' should be defined by you and should return some performance metric
# Here, we mock this by assuming it returns accuracy
reduced_performance = evaluate_model(reduced_X, y)
print("Performance with reduced features:", reduced_performance)


In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GlobalAveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

balanced_acc_scores = []
mcc_scores = []
f1_scores = []
recall_scores = []


def create_simple_encoder():
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length),
        GlobalAveragePooling1D()
    ])
    
    return model

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)
max_sequence_length = max(len(seq) for seq in X_sequences)
X_padded = pad_sequences(X_sequences, maxlen=max_sequence_length, padding='post')

# Define vocabulary size and embedding dimension
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 200

In [ ]:
for fold_idx, (train_index, test_index) in enumerate(stratified_kfold_resampled.split(X_padded, y_encoded), 1):
    X_train_fold, X_test_fold = X_padded[train_index], X_padded[test_index]
    y_train_fold, y_test_fold = y_encoded[train_index], y_encoded[test_index]

    # Create and compile the simple encoder model
    simple_encoder = create_simple_encoder()
    simple_encoder.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the simple encoder on the training data
    simple_encoder.fit(X_train_fold, y_train_fold, epochs=5, batch_size=32, verbose=0)

    # Get embeddings for the training and test data
    X_train_embeddings_fold = simple_encoder.predict(X_train_fold)
    X_test_embeddings_fold = simple_encoder.predict(X_test_fold)

    # Resample the training data
    X_train_resampled_fold, y_train_resampled_fold = smote_enn.fit_resample(X_train_embeddings_fold, y_train_fold)

    # Perform grid search on the entire resampled training data
    grid_search.fit(X_train_resampled_fold, y_train_resampled_fold)

    # Get the best hyperparameters from grid search
    best_params = grid_search.best_params_
    best_logistic_classifier = grid_search.best_estimator_

    # Train the best SVM model on the entire resampled training data
    best_logistic_classifier.fit(X_train_resampled_fold, y_train_resampled_fold)

    # Predict labels on the test fold
    y_pred_fold = best_logistic_classifier.predict(X_test_embeddings_fold)

    # Calculate evaluation metrics for this fold
    balanced_acc_fold = balanced_accuracy_score(y_test_fold, y_pred_fold)
    mcc_fold = matthews_corrcoef(y_test_fold, y_pred_fold)
    f1_fold = f1_score(y_test_fold, y_pred_fold)
    recall_fold = recall_score(y_test_fold, y_pred_fold,average='weighted')

    # Append the scores to the lists
    balanced_acc_scores.append(balanced_acc_fold)
    mcc_scores.append(mcc_fold)
    f1_scores.append(f1_fold)
    recall_scores.append(recall_fold)

    # Print the evaluation metrics for this fold
    print(f"Fold {fold_idx}:")
    print(f"Best Hyperparameters: {best_params}")
    print(f"Balanced Accuracy: {balanced_acc_fold:.2f}")
    print(f"F1-Score: {f1_fold:.2f}")
    print(f"Matthew's Correlation Coefficient: {mcc_fold:.2f}")
    print(f"Recall: {recall_fold:.2f}")
    print()

# Print average scores
print("Average Scores simple encoder:")
print(f"{np.mean(balanced_acc_scores):.2f}")
print(f"{np.mean(f1_scores):.2f}")
print(f"{np.mean(mcc_scores):.2f}")
print(f"{np.mean(recall_scores):.2f}")
